---
title: "PARI/GP in OCaml: Implementing and Testing a Modular Discrete Log Solver"
description: "I'll demonstrate how to use OCaml bindings to the [PARI/GP](https://pari.math.u-bordeaux.fr/) number theory library I've written."
date: "12/23/2023"
categories:
  - ocaml
  - algebra
---

# Overview

This notebook aims to showcase how functional programming and property-based testing allow for readable and correct numerical algorithms, using the [PARI/GP](https://pari.math.u-bordeaux.fr/) number theory package. In particular, it aims to demonstrate the effectiveness of [strong static typing](https://en.wikipedia.org/wiki/Strong_and_weak_typing) to avoid invalid operations between incompatible values, [polymorphic types](https://www.cs.princeton.edu/~dpw/courses/cos326-12/notes/polymorphism.php) for writing generic algorithms, [higher-order functions](https://v2.ocaml.org/releases/5.1/manual/coreexamples.html#s%3Afunctions-as-values) for composing algorithms, and type inference to avoid cluttering code with complex types. You can download the notebook, and run it on your end using the [OCaml Jupyter kernel](https://github.com/akabe/ocaml-jupyter) as well as installing the following OCaml libraries:

* [pari](https://ocaml.org/p/pari/latest): bindings to PARI/GP
* [iter](https://ocaml.org/p/iter/latest): functions to iterate on collections, we'll use this to write loops in the functional style
* [hacl-star](https://ocaml.org/p/hacl-star/latest): we'll use the hash function SHA3 from the HACL* project
* [qcheck](https://ocaml.org/p/qcheck/latest): property-based testing library

(With [OPAM](https://opam.ocaml.org/): `opam install pari iter hacl-star qcheck`.)

# Why OCaml?
Currently, one can use the GP scripting language or [Python bindings](https://github.com/sagemath/cypari2) to PARI, the latter being used by [Sagemath](https://www.sagemath.org/). A drawback is that the C, GP, and Python APIs are loosely typed, allowing syntactically correct but semantically incorrect mathematical operations, though runtime exceptions can sometimes occur in such instances. By contrast, the OCaml bindings add a thin layer of static typing to prevent such invalid operations by giving precise domains to the functions. Moreover, static types can help document functions, to better understand how to use them: it is not clear how to compute the [Smith Normal Form of a matrix with number field coefficients](https://pari.math.u-bordeaux.fr/dochtml/html-stable/General_number_fields.html#nfsnf) from the documentation, for example. More generally, OCaml is a high-level language with decent performance focusing on correctness and grounded in mathematical formalism (lambda calculus), making it quite appealing in the context of mathematical algorithms.

# Case Study: Implementing a Modular Discrete Log Solver

## The Pohlig-Hellman algorithm

In [15]:
#require "pari,iter,hacl-star,qcheck"

First, we'll import and put all functions from the PARI library into scope:

In [16]:
open Pari

(* This line is mandatory to initialize the PARI stack, heap,
   table of primes... *)
let () = pari_init 2_000_000_000 (Unsigned.ULong.of_int 500_000)

(* Computes f() then resets the stack pointer to its original state,
   freeing up memory. *)
let clean_stack f = let sp = get_avma () in let _ = f () in set_avma sp

val clean_stack : (unit -> 'a) -> unit = <fun>


The [Pohlig-Hellman algorithm](https://en.wikipedia.org/wiki/Pohlig%E2%80%93Hellman_algorithm) is a divide-and-conquer algorithm that solves discrete logarithms on finite abelian groups by taking advantage of the smoothness of their orders. It breaks down the computation of a discrete logarithm into projections in the subgroups, then recombines the logarithms of the projections using the Chinese Remainder Theorem.

The algorithm handles the computation of discrete logarithms in subgroups of prime power order [a little differently](https://en.wikipedia.org/wiki/Pohlig%E2%80%93Hellman_algorithm#Groups_of_prime-power_order). We let the subroutine `dlp_solve_prime` fail with `None` (for instance if no discrete logarithm exists when the input group element is not in the subgroup generated by the base, or if the function did not terminate within a reasonable timeframe), so we chain the intermediate results using the custom let binding `let*?`.

In [17]:
let ( let*? ) = Option.bind

(** Solves the discrete log of h to the given base where the order of the group
    is prime^valuation. *)
let pohlig_hellman_prime_power_order ~mul ~pow ~dlp_solve_prime ~base
    ~prime ~base_order_factorization h =
  let prime_power (p, _) = Integer.equal p prime in
  match Array.find_opt prime_power base_order_factorization with
  | None -> Some (Integer.of_int 0)
  | Some (prime, valuation) ->
      let proj_base = pow base Integer.(pow prime (of_int (valuation - 1))) in
      let f x k =
        let*? x = x in
        let*? log =
          dlp_solve_prime ~base:proj_base ~prime
            (pow
               (mul (pow base (Integer.neg x)) h)
               (Integer.pow prime (Integer.of_int (valuation - k - 1))))
        in
        Some Integer.(add x (mul (pow prime (of_int k)) log))
      in
      Iter.(fold f (Some (Integer.of_int 0)) (0 -- (valuation - 1)))

val ( let*? ) : 'a option -> ('a -> 'b option) -> 'b option = <fun>


val pohlig_hellman_prime_power_order :
  mul:('a -> 'b -> 'c) ->
  pow:('c -> Pari.Integer.t -> 'a) ->
  dlp_solve_prime:(base:'a ->
                   prime:Pari.Integer.t -> 'a -> Pari.Integer.t option) ->
  base:'c ->
  prime:Pari.Integer.t ->
  base_order_factorization:(Pari.Integer.t * int) array ->
  'b -> Pari.Integer.t option = <fun>


OCaml inferred for us the signature of the function. Note that it is the most general type that could be given to this function satisfying the constraints from the function definition. In particular, it does not exclusively use PARI types (the generic types `'a`, `'b`, `'c` are used instead). Then comes the main routine:

In [18]:
let pohlig_hellman ~order ~mul ~pow ~dlp_solve_prime ~base ~group_order h =
  let f base_order_factorization (prime, valuation) =
    let ord = Integer.(pow prime (of_int valuation)) in
    let n = Integer.(divexact (to_integer group_order) ord) in
    let*? intermediate_log =
      pohlig_hellman_prime_power_order ~mul ~pow ~dlp_solve_prime
        ~base:(pow base n) ~prime (pow h n)
        ~base_order_factorization
    in
    Some (Integer_mod.create intermediate_log ~modulo:ord)
  in
  let main () =
    if gequal (pow base (Integer.zero ())) h then Some (Integer.zero ())
    else if gequal base h then Some (Integer.of_int 1)
    (* If the order of [h] does not divide the order of the base, then
       no log exists. It is a necessary but not a sufficient condition
       for the existence of a log: h could be in a different subgroup
       that has the same order as the one generated by base.  *)
    else if (dvdii (order base) (order h) = 0) then None else
    let factors = factor group_order in
    let base_order = Integer.inj_unique_factorization_domain (order base) in
    let logs = Array.map (f (factor base_order)) factors in
    if Array.for_all Option.is_some logs then
      let logs = Vector.of_array (Array.map Option.get logs) in
      Some Integer_mod.(lift (chinese logs))
    else None
  in
  with_stack_clean_opt main

val pohlig_hellman :
  order:(('a, 'b) Pari.typ -> Pari.Integer.t) ->
  mul:(('a, 'b) Pari.typ -> ('a, 'b) Pari.typ -> ('a, 'b) Pari.typ) ->
  pow:(('a, 'b) Pari.typ -> Pari.Integer.t -> ('a, 'b) Pari.typ) ->
  dlp_solve_prime:(base:('a, 'b) Pari.typ ->
                   prime:Pari.Integer.t ->
                   ('a, 'b) Pari.typ -> Pari.Integer.t option) ->
  base:('a, 'b) Pari.typ ->
  group_order:(Pari.integer, Pari.unique_factorization_domain) Pari.typ ->
  ('a, 'b) Pari.typ -> (Pari.integer, Pari.ring) Pari.typ option = <fun>


The `mul` function has a more restrictive type here `mul:('a -> 'a -> 'a)` than in `pohlig_Hellman_prime_power_order` where it is `mul:('a -> 'b -> 'c)`, which is expected. OCaml derived the valid type for this function for us!

## Pollard's Rho Algorithm

The Pollard's Rho algorithm computes discrete logarithms in cyclic groups. It applies the [Floyd cycle-finding algorithm](https://en.wikipedia.org/wiki/Cycle_detection#Floyd's_tortoise_and_hare) and as such has a constant memory footprint. First, we need this auxiliary function to compute the discrete logarithm when a collision is detected between the "tortoise" and "hare" sequences:

In [19]:
let[@inline] find_solution a b ~group_order ~base ~x ~pow =
  (* solves for x the congruence a * x = b mod n *)
  let[@inline] solve_congruence a b ~group_order =
    let gcd', s, _ = Integer.gcdext a group_order in
    if Integer.(equal (modulo b gcd') (of_int 0)) then
      let q = Integer.divexact b gcd' in
      let r = Integer.divexact group_order gcd' in
      let qs = Integer.Infix.(q * s) in
      Some Iter.(map
            (fun i -> Integer.(Infix.(qs + (of_int i * r))))
            (* gcd' < group_order, and is assumed to fit in an OCaml integer *)
            (0 -- (Integer.to_int gcd' - 1)))
    else None
  in
  let*? candidates = solve_congruence a b ~group_order in
  let is_solution n = if gequal (pow base n) x then Some n else None in
  let*? solution = Iter.find is_solution candidates in
  Some (Integer.modulo solution group_order)

val find_solution :
  Pari.Integer.t ->
  Pari.Integer.t ->
  group_order:Pari.Integer.t ->
  base:'a ->
  x:('b, 'c) Pari.typ ->
  pow:('a -> Pari.Integer.t -> ('b, 'c) Pari.typ) -> Pari.Integer.t option =
  <fun>


For better performance, the PARI library allocates most of the time data on a stack instead of a heap. Therefore, we must manually clean the stack when some variables aren't needed. This can be achieved by wrapping a computed value `v : ('kind, 'structure) Pari.typ` in anonymous functions `fun () -> v` that are passed to `with_stack_clean*` functions. Once a function from the `with_stack_clean*` family terminates, all temporary values allocated by the anonymous function are deallocated and the result of the computation `v` is returned. Warning! OCaml supports strict evaluation by default, so one has to put the actual computation (chain of `let` bindings) inside the closure `fun () -> ...`! 

In [20]:
let rho_pollard ~one ~mul ~pow ?start ~class_x ~group_order ~base x =
  let[@inline] f y =
    match class_x y with 1 -> mul base y | 0 -> mul y y | _ -> mul x y
  in
  with_stack_clean_opt (fun () ->
  let[@inline] g x k =
    Integer.(Infix.(match class_x x with
                    | 0 -> of_int 2 * k mod group_order
                    | 1 -> (k + of_int 1) mod group_order
                    | _ -> k))
  in
  let[@inline] h x k =
    Integer.(Infix.(match class_x x with
                  | 0 -> of_int 2 * k mod group_order
                  | 1 -> k
                  | _ -> (k + of_int 1) mod group_order))
  in
  let rec loop (x_k, a_k, b_k, x_2k, a_2k, b_2k) =
    let x_k, a_k, b_k, x_2k, a_2k, b_2k =
      (* if the loop is very long the PARI stack must be
          cleaned to keep here the constant memory footprint *)
      with_stack_clean6 (fun () ->
          ( f x_k,
            g x_k a_k,
            h x_k b_k,
            f (f x_2k),
            g (f x_2k) (g x_2k a_2k),
            h (f x_2k) (h x_2k b_2k) ))
    in
    if gequal x_k x_2k then
      Integer.Infix.(
        find_solution (b_2k - b_k) (a_k - a_2k) ~group_order ~base ~x ~pow)
    else loop (x_k, a_k, b_k, x_2k, a_2k, b_2k)
  in
  match start with
  | Some s -> loop s
  | None ->
      let a, b = Integer.(of_int 0, of_int 0) in
      loop (one, a, b, one, a, b))

val rho_pollard :
  one:('a, 'b) Pari.typ ->
  mul:(('a, 'b) Pari.typ -> ('a, 'b) Pari.typ -> ('a, 'b) Pari.typ) ->
  pow:(('a, 'b) Pari.typ -> Pari.Integer.t -> ('a, 'b) Pari.typ) ->
  ?start:('a, 'b) Pari.typ * Pari.Integer.t * Pari.Integer.t *
         ('a, 'b) Pari.typ * Pari.Integer.t * Pari.Integer.t ->
  class_x:(('a, 'b) Pari.typ -> int) ->
  group_order:Pari.Integer.t ->
  base:('a, 'b) Pari.typ ->
  ('a, 'b) Pari.typ -> (Pari.integer, Pari.ring) Pari.typ option = <fun>


The function `gequal` is generic in that it works with any PARI values _of the same subtype_ given by the phantom type parameter `('kind, 'structure)`:

In [21]:
gequal

- : ('kind, 'structure) Pari.typ -> ('kind, 'structure) Pari.typ -> bool =
<fun>


This helper function retries Pollard's Rho with different starting values:

In [22]:
let rho_pollard_with_retries ~one ~mul ~pow ~class_x ~group_order ~base h =
  let f ?start () =
    rho_pollard ?start ~one ~mul ~pow ~class_x ~group_order ~base h
  in
  let start ord =
    let a_k = Integer.random ord in
    let b_k = Integer.random ord in
    let x_k = mul (pow base a_k) (pow h b_k) in
    (x_k, a_k, b_k, x_k, a_k, b_k)
  in
  let rec loop () =
    match f ~start:(start group_order) () with
    | Some res -> Some res
    | None -> loop ()
  in
  match f () with Some res -> Some res | None -> loop ()

val rho_pollard_with_retries :
  one:('a, 'b) Pari.typ ->
  mul:(('a, 'b) Pari.typ -> ('a, 'b) Pari.typ -> ('a, 'b) Pari.typ) ->
  pow:(('a, 'b) Pari.typ -> Pari.Integer.t -> ('a, 'b) Pari.typ) ->
  class_x:(('a, 'b) Pari.typ -> int) ->
  group_order:Pari.Integer.t ->
  base:('a, 'b) Pari.typ ->
  ('a, 'b) Pari.typ -> (Pari.integer, Pari.ring) Pari.typ option = <fun>


# Testing Instantiations of the Discrete Log Solver

## Instantiating over $\mathbf{F}_p^\times$

We now specialize the pohlig_hellman function to work with prime field elements (for small discrete log instances, we use the `Integer_mod.log` function):

In [23]:
let zn_dlog ~base x =
  with_stack_clean_opt (fun () ->
    let modulo = Integer_mod.get_modulo base in
    let group_order = Integer.inj_unique_factorization_domain (eulerphi modulo) in
    let class_x x = Integer.(to_int (modulo (Integer_mod.lift x) (of_int 3))) in
    let mul = Integer_mod.mul in
    let pow = Integer_mod.pow in
    let dlp_solve_prime ~base ~prime =
      if cmpii prime (Integer.of_int 20) > 0 then
        rho_pollard_with_retries ~one:base ~mul ~pow ~class_x ~group_order:prime
          ~base
      else Integer_mod.log ~base
    in
    let*? x =
      pohlig_hellman ~order:Integer_mod.order ~pow ~mul ~dlp_solve_prime ~base
        ~group_order x
    in
    Some (Integer.modulo x Integer_mod.(order base)))

val zn_dlog :
  base:(Pari.integer_mod, 'a) Pari.typ ->
  (Pari.integer_mod, 'a) Pari.typ ->
  (Pari.integer, Pari.ring) Pari.typ option = <fun>


## Instantiating over $E(\mathbf{F})$

And here is the specialized code for elliptic curves. Not necessarily defined over finite fields, as evidenced by the function's type `ell:('a, field) typ Elliptic_curve.t ->
base:('a, field) typ Elliptic_curve.elt ->
('a, field) typ Elliptic_curve.elt ->
(integer, ring) typ option`. However, such generality is not needed here as the algorithms from this blog post require finite and cyclic groups, which we could reflect in the signature.

In [24]:
let ell_solve_dlog ~ell ~base x =
  with_stack_clean_opt (fun () ->
      let class_x x =
        let h =
          Integer.of_hex
            (Hex.of_bytes (Hacl_star.Hacl.SHA3_512.hash (gentobytes x)))
        in
        Integer.(to_int (modulo h (of_int 3)))
      in
      let group_order =
        Integer.inj_unique_factorization_domain
          (Elliptic_curve.order_elt ell base)
      in
      let mul = Elliptic_curve.add ell in
      let pow p n = Elliptic_curve.mul ell ~n ~p in
      pohlig_hellman
        ~order:(Elliptic_curve.order_elt ell)
        ~pow ~mul
        ~dlp_solve_prime:(fun ~base ~prime ->
          rho_pollard_with_retries ~one:base ~mul ~pow ~class_x
            ~group_order:prime ~base)
        ~base ~group_order x)

val ell_solve_dlog :
  ell:('a, Pari.field) Pari.typ Pari.Elliptic_curve.t ->
  base:('a, Pari.field) Pari.typ Pari.Elliptic_curve.elt ->
  ('a, Pari.field) Pari.typ Pari.Elliptic_curve.elt ->
  (Pari.integer, Pari.ring) Pari.typ option = <fun>


## Property-Based Tests

You can refer to QCheck's [documentation](https://c-cube.github.io/qcheck/0.21/qcheck-core/QCheck2/index.html) to understand the tests from this section. We define first generators for the values at hand, and printers to show the values for which the test fails:

In [25]:
let option_eq = Option.equal gequal

let random_int bound = QCheck2.Gen.(1 -- (bound - 1))

let random_integer bound =
  let open QCheck2.Gen in
  let+ n = random_int (Integer.to_int bound) in
  Integer.of_int n

let random_prime =
  let open QCheck2.Gen in
  let* bits_amount = QCheck2.Gen.(9 -- 10) in
  let+ res = return (Integer.random_prime ~bits_amount) in
  if gequal res (Integer.of_int 2) then Integer.of_int 3 else res

let random_znelt modulo =
  let open QCheck2.Gen in
  let* x = random_integer modulo in
  return Integer_mod.(inj_group (create x ~modulo))

let random_znlog_instance =
  let open QCheck2.Gen in
  let* modulo = random_prime in
  let base = Integer_mod.(inj_group (create (Integer.of_int 2) ~modulo)) in
  let* x = random_znelt modulo in
  return (base, x)

let print_random_znlog_instance =
  QCheck2.Print.(
    comap
      (fun (base, x) -> (Integer_mod.to_string base, Integer_mod.to_string x))
      (tup2 string string))

let random_ell_elt ell ~base ~order =
  let open QCheck2.Gen in
  let* n = random_int order in
  return (Elliptic_curve.mul ell ~p:base ~n:(Integer.of_int n))

let random_ell =
  let open QCheck2.Gen in
  let* p = random_prime in
  let* x = random_integer p in
  let rec loop () =
    let* y = random_integer p in
    match
      Elliptic_curve.create
        ~a4:(Integer_mod.create_assume_prime_modulus x ~modulo:p)
        ~a6:(Integer_mod.create_assume_prime_modulus y ~modulo:p)
        ()
    with
    | Some ell ->
        let order = Elliptic_curve.order ell in
        if gequal order (Integer.of_int 1) then loop () else return ell
    | None -> loop ()
  in
  loop ()

let random_elldlog_instance =
  let open QCheck2.Gen in
  let* ell = random_ell in
  let order = Integer.to_int (Elliptic_curve.order ell) in
  let base = Elliptic_curve.random ell in
  let* x = random_ell_elt ell ~base ~order in
  return (ell, base, x)

let print_random_elldlog_instance =
  QCheck2.Print.(
    comap
      (fun (ell, base, x) ->
        ( Elliptic_curve.to_string ell,
          Elliptic_curve.to_string_elt base,
          Elliptic_curve.to_string_elt x ))
      (tup3 string string string))

val option_eq :
  ('_weak6, '_weak7) Pari.typ option ->
  ('_weak6, '_weak7) Pari.typ option -> bool = <fun>


val random_int : int -> int QCheck2.Gen.t = <fun>


val random_integer : Pari.Integer.t -> Pari.Integer.t QCheck2.Gen.t = <fun>


val random_prime : Pari.Integer.t QCheck2.Gen.t = <abstr>


val random_znelt :
  Pari.Integer.t -> (Pari.integer_mod, Pari.group) Pari.typ QCheck2.Gen.t =
  <fun>


...

val print_random_znlog_instance :
  ((Pari.integer_mod, '_weak8) Pari.typ *
   (Pari.integer_mod, '_weak9) Pari.typ)
  QCheck2.Print.t = <fun>


val random_ell_elt :
  ('a, Pari.field) Pari.typ Pari.Elliptic_curve.t ->
  base:('a, Pari.field) Pari.typ Pari.Elliptic_curve.elt ->
  order:int ->
  ('a, Pari.field) Pari.typ Pari.Elliptic_curve.elt QCheck2.Gen.t = <fun>


val random_ell :
  (Pari.integer_mod, Pari.field) Pari.typ Pari.Elliptic_curve.t QCheck2.Gen.t =
  <abstr>


val random_elldlog_instance :
  ((Pari.integer_mod, Pari.field) Pari.typ Pari.Elliptic_curve.t *
   (Pari.integer_mod, Pari.field) Pari.typ Pari.Elliptic_curve.elt *
   (Pari.integer_mod, Pari.field) Pari.typ Pari.Elliptic_curve.elt)
  QCheck2.Gen.t = <abstr>


val print_random_elldlog_instance :
  (('_weak10, Pari.field) Pari.typ Pari.Elliptic_curve.t *
   ('a, Pari.field) Pari.typ Pari.Elliptic_curve.elt *
   ('b, Pari.field) Pari.typ Pari.Elliptic_curve.elt)
  QCheck2.Print.t = <fun>


And finally the property to be tested, we compare the result against the one from the `Integer_mod.log` function provided by PARI:

In [26]:
let zn_dlog_test =
  QCheck2.Test.make ~count:10000 ~name:"zn_dlog"
    ~print:print_random_znlog_instance random_znlog_instance
    (fun (base, x) ->
      let log' = Integer_mod.log ~base x in
      let log = zn_dlog ~base x in
      option_eq log log')

let ell_dlog_test =
  QCheck2.Test.make ~count:5000 ~name:"ell_dlog"
    ~print:print_random_elldlog_instance random_elldlog_instance
    (fun (ell, base, x) ->
      let log' = Elliptic_curve.log ell ~base x in
      let log = ell_solve_dlog ~ell ~base x in
      option_eq log log')

let () = clean_stack (fun () ->
  QCheck_runner.run_tests ~verbose:true [ zn_dlog_test; ell_dlog_test ])

val zn_dlog_test : QCheck2.Test.t = QCheck2.Test.Test <abstr>


val ell_dlog_test : QCheck2.Test.t = QCheck2.Test.Test <abstr>


generated error  fail  pass / total     time test name
[✓] 10000     0     0 10000 / 10000     3.3s zn_dlogg)
[✓]  5000     0     0  5000 /  5000    16.9s ell_dlogg)
success (ran 2 tests)


All this could be achieved with the GP scripting language or Python, as they both support higher-order functions. Python even has property-based testing libraries. What sets OCaml apart is its advanced type system: phantom type parameters allow for genericity and a form of subtyping, static types document functions and provide usage hints, type inference can guide the programmer in the writing of functions.